In [1]:
pip install requests deepgram-sdk openai

                                              0.0/98.2 kB ? eta -:--:--
     -------------------------------------    92.2/98.2 kB 2.6 MB/s eta 0:00:01
     -------------------------------------    92.2/98.2 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 98.2/98.2 kB 514.5 kB/s eta 0:00:00
                                              0.0/328.8 kB ? eta -:--:--
     ----------------                       143.4/328.8 kB 4.2 MB/s eta 0:00:01
     ----------------------                 194.6/328.8 kB 2.0 MB/s eta 0:00:01
     -----------------------------          256.0/328.8 kB 1.7 MB/s eta 0:00:01
     -------------------------------------  327.7/328.8 kB 1.8 MB/s eta 0:00:01
     -------------------------------------  327.7/328.8 kB 1.8 MB/s eta 0:00:01
     -------------------------------------  327.7/328.8 kB 1.8 MB/s eta 0:00:01
     -------------------------------------  327.7/328.8 kB 1.8 MB/s eta 0:00:01
     -------------------------------------  327.7/328.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2024.3.1 which is incompatible.
ypy-websocket 0.8.2 requires aiofiles<23,>=22.1.0, but you have aiofiles 24.1.0 which is incompatible.


In [1]:
import requests
from deepgram import Deepgram
import openai

In [2]:
class SpeechService:
    def __init__(self, stt_engine, tts_engine, llm_engine, stt_api_key, tts_api_key, llm_api_key, system_prompt):
        self.stt_engine = stt_engine
        self.tts_engine = tts_engine
        self.llm_engine = llm_engine
        self.stt_api_key = stt_api_key
        self.tts_api_key = tts_api_key
        self.llm_api_key = llm_api_key
        self.system_prompt = system_prompt
        
        # Initialize STT engine
        if self.stt_engine == 'Deepgram':
            self.stt_client = Deepgram(self.stt_api_key)
        else:
            raise ValueError('Unsupported STT engine')

        # Initialize TTS engine
        if self.tts_engine == 'Deepgram':
            self.tts_client = Deepgram(self.tts_api_key)
        elif self.tts_engine == 'OpenAI':
            openai.api_key = self.tts_api_key
        else:
            raise ValueError('Unsupported TTS engine')

        # Initialize LLM engine
        if self.llm_engine == 'OpenAI':
            openai.api_key = self.llm_api_key
        else:
            raise ValueError('Unsupported LLM engine')

    def transcribe_audio(self, audio_path):
        if self.stt_engine == 'Deepgram':
            with open(audio_path, 'rb') as audio_file:
                response = self.stt_client.transcription.prerecorded(audio_file, {'punctuate': True})
                return response.get('results', [])[0].get('alternatives', [])[0].get('transcript', '')

    def synthesize_text(self, text):
        if self.tts_engine == 'Deepgram':
            response = self.tts_client.text_to_speech.synthesize(text)
            return response['audio_url']
        elif self.tts_engine == 'OpenAI':
            response = openai.Completion.create(engine="davinci", prompt=text)
            return response.choices[0].text.strip()

    def generate_text(self, prompt):
        response = openai.Completion.create(engine="davinci", prompt=self.system_prompt + '\n\n' + prompt)
        return response.choices[0].text.strip()

In [5]:
stt_api_key = 'abebde1f8781c240a32d9e508f58ccd1400e64db'
tts_api_key = 'e92684c8b806bc18e7fd724b0c98f61d376fa4dd'  # Can be Deepgram or OpenAI API key
llm_api_key = 'sk-proj-3OR9WxrXPafjoAXRf3uST3BlbkFJewlZgiMxJLIbbHKe4BTY'

# Define the system prompt for the LLM
system_prompt = "You are a helpful assistant."

# Initialize the SDK
sdk = SpeechService(
    stt_engine='Deepgram',
    tts_engine='OpenAI',
    llm_engine='OpenAI',
    stt_api_key=stt_api_key,
    tts_api_key=tts_api_key,
    llm_api_key=llm_api_key,
    system_prompt=system_prompt
)

# Example usage
audio_path = 'harvard.wav'
transcription = sdk.transcribe_audio(audio_path)
print("Transcription:", transcription)

synthesized_speech = sdk.synthesize_text("Hello, how can I help you?")
print("Synthesized Speech URL:", synthesized_speech)

generated_text = sdk.generate_text("Tell me a joke.")
print("Generated Text:", generated_text)

AttributeError: 'coroutine' object has no attribute 'get'

In [6]:
pip install deepgram-sdk==2.12.0

  Attempting uninstall: deepgram-sdk
    Found existing installation: deepgram-sdk 3.4.0
    Uninstalling deepgram-sdk-3.4.0:
      Successfully uninstalled deepgram-sdk-3.4.0
Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests
from deepgram import Deepgram
import openai
import asyncio
import nest_asyncio

# Apply the patch for nested asyncio
nest_asyncio.apply()

class SpeechService:
    def __init__(self, stt_engine, tts_engine, llm_engine, stt_api_key, tts_api_key, llm_api_key, system_prompt):
        self.stt_engine = stt_engine
        self.tts_engine = tts_engine
        self.llm_engine = llm_engine
        self.stt_api_key = stt_api_key
        self.tts_api_key = tts_api_key
        self.llm_api_key = llm_api_key
        self.system_prompt = system_prompt
        
        # Initialize STT engine
        if self.stt_engine == 'Deepgram':
            self.stt_client = Deepgram(self.stt_api_key)
        else:
            raise ValueError('Unsupported STT engine')

        # Initialize TTS engine
        if self.tts_engine == 'Deepgram':
            self.tts_client = Deepgram(self.tts_api_key)
        elif self.tts_engine == 'OpenAI':
            openai.api_key = self.tts_api_key
        else:
            raise ValueError('Unsupported TTS engine')

        # Initialize LLM engine
        if self.llm_engine == 'OpenAI':
            openai.api_key = self.llm_api_key
        else:
            raise ValueError('Unsupported LLM engine')

    async def transcribe_audio(self, audio_path):
        if self.stt_engine == 'Deepgram':
            with open(audio_path, 'rb') as audio_file:
                source = {'buffer': audio_file, 'mimetype': 'audio/wav'}
                response = await self.stt_client.transcription.prerecorded(source, {'punctuate': True})
                print("Response:", response)  # Debug print

                # Navigate through the response structure
                results = response.get('results', {})
                channels = results.get('channels', [])
                if channels:
                    alternatives = channels[0].get('alternatives', [])
                    if alternatives:
                        return alternatives[0].get('transcript', '')
        return ''  # Default return if no transcript is found

    def synthesize_text(self, text):
        if self.tts_engine == 'Deepgram':
            response = self.tts_client.text_to_speech.synthesize(text)
            return response.get('audio_url', '')
        elif self.tts_engine == 'OpenAI':
            response = openai.Completion.create(engine="davinci", prompt=text)
            return response.choices[0].text.strip()

    def generate_text(self, prompt):
        response = openai.Completion.create(engine="davinci", prompt=self.system_prompt + '\n\n' + prompt)
        return response.choices[0].text.strip()

# Replace these with your actual API keys
stt_api_key = 'abebde1f8781c240a32d9e508f58ccd1400e64db'
tts_api_key = 'e92684c8b806bc18e7fd724b0c98f61d376fa4dd'  # Can be Deepgram or OpenAI API key
llm_api_key = 'sk-proj-3OR9WxrXPafjoAXRf3uST3BlbkFJewlZgiMxJLIbbHKe4BTY'

# Define the system prompt for the LLM
system_prompt = "You are a helpful assistant."

# Initialize the SDK
sdk = SpeechService(
    stt_engine='Deepgram',
    tts_engine='Deepgram',
    llm_engine='OpenAI',
    stt_api_key=stt_api_key,
    tts_api_key=tts_api_key,
    llm_api_key=llm_api_key,
    system_prompt=system_prompt
)

# Example usage
async def main():
    audio_path = 'harvard.wav'
    transcription = await sdk.transcribe_audio(audio_path)
    print("Transcription:", transcription)

    synthesized_speech = sdk.synthesize_text("Hello, how can I help you?")
    print("Synthesized Speech URL:", synthesized_speech)

    generated_text = sdk.generate_text("Tell me a joke.")
    print("Generated Text:", generated_text)

# Run the example
await main()


Response: {'metadata': {'transaction_key': 'deprecated', 'request_id': '0def6f57-c1a5-441f-8803-d33c47564397', 'sha256': '971b4163670445c415c6b0fb6813c38093409ecac2f6b4d429ae3574d24ad470', 'created': '2024-07-22T10:17:01.212Z', 'duration': 18.356188, 'channels': 1, 'models': ['1ed36bac-f71c-4f3f-a31f-02fd6525c489'], 'model_info': {'1ed36bac-f71c-4f3f-a31f-02fd6525c489': {'name': 'general', 'version': '2024-01-26.8851', 'arch': 'base'}}}, 'results': {'channels': [{'alternatives': [{'transcript': 'The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. A Tacos All pasteur are my favorite. A zest food is the hot cross bun.', 'confidence': 0.9897461, 'words': [{'word': 'the', 'start': 1.318125, 'end': 1.6376704, 'confidence': 0.9951172, 'punctuated_word': 'The'}, {'word': 'stale', 'start': 1.6376704, 'end': 2.1169887, 'confidence': 0.55810547, 'punctuated_word': 'stale'}, {'word': 'smell', 'start': 2

AttributeError: 'Deepgram' object has no attribute 'text_to_speech'

In [8]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: nest_asyncio in c:\users\shahana s\anaconda3\lib\site-packages (1.5.6)



In [4]:
pip install openai==0.28

                                              0.0/76.5 kB ? eta -:--:--
     -------------------------------------    71.7/76.5 kB ? eta -:--:--
     -------------------------------------- 76.5/76.5 kB 850.1 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.36.1
    Uninstalling openai-1.36.1:
      Successfully uninstalled openai-1.36.1
Note: you may need to restart the kernel to use updated packages.
